<center><img src="http://i.imgur.com/sSaOozN.png" width="500"></center>

# Data Integration in Python


## Course: Computational Thinking for Governance Analytics<br> Winter 2017

### Prof. José Manuel Magallanes, PhD 

When you are done cleaning and formatting your data, you need to integrate it, as, most of the time, you were doing that in different files. Let me get some files first.

Let's use these data on [prisons](http://www.doc.wa.gov/information/data/docs/admissions-releases-by-county.pdf) from the State of Washington. The data is in PDF, which is always difficult to process. I tried two interesting services and both failed ([openRefine](http://openrefine.org/) and [tabula](http://tabula.technology/)). This one did a pretty gob job: [pdfTables](https://pdftables.com/). The created file is 'countiesJail.csv'.

In [ ]:
import os
import pandas as pd
folder="data"
fileName="countiesJail.csv"
fileToRead=os.path.join(folder,fileName)
dataJail=pd.read_csv(fileToRead,skiprows=2)

In [ ]:
dataJail.head(10)

Let's check the bottom:

In [ ]:
dataJail.tail(10)

The only thing clear is that I have rows I do not need in the tail. Let's subset getting rid of the last seven rows:

In [ ]:
dataJail=dataJail.head(-7)

Now let's work on the headers. From above, you understand that each column informs the amount of people released and admitted per year. However, that is shown in two rows. There is no easy way to use those rows and make a new header row. So I am going to create the headers. I need to create something like:  "Admission2006", "Release2006", "Admission2007", "Release2007",, etc. This is how you do that:

In [ ]:
years=list(range(2006,2016))
types=['Admission', 'Release']

# first element
titlesNew=['county']

# adding the other elements
titlesNew=titlesNew + [t + str(y) for y in years for t in types]

# this is a list comprehension: [t + str(y) for y in years for t in types]

## see result:
titlesNew

Let's simply change the headers now:

In [ ]:
dataJail.columns=titlesNew

# a quick look:
dataJail.head()

You see that the first row should dissappear:

In [ ]:
dataJail=dataJail.tail(-1)

When you get rid of a row that may affect the row names in the data frame you keepin on working, it is better to reset the indexes (positions):

In [ ]:
dataJail.reset_index(drop=True,inplace=True)
# You always want to use 'drop=True'
# When you use 'inplace=True', you do not need to do this:
# dataJail=dataJail.reset_index(drop=True)

In [ ]:
# then
dataJail.head()

These columns represent counts, let's see what they are for Python:

In [ ]:
dataJail.dtypes

When python has numbers, it should say _float_; when it says _object_, it means there are strings in the columns.

In [ ]:
# you can not identify visually:
dataJail.Admission2015

How to be sure about the symbol that is provoking that numbers be read as strings?

In [ ]:
# using regex
import re
pattern='\\d' #any number
nothing=''

# ".columns" gives you all the column names.
for column in dataJail.columns[1:]:  # "[1:]" avoids the first elemnt (0)
    for element in dataJail[column]: 
        result=re.sub(pattern,nothing,element) #delete any number you find an replace it with...
        if result!=nothing:
            print(result)

The idea is that you know if you have one particular improper character or not, you may find more, and each may require a different treatment. In this case, we are just gonna supress the comma and then convert the value into _float_ (numeric in R): 

In [ ]:
# replace ',' with nothing, and convert the column to number:
toNum= lambda x: x.str.replace(',','').astype('float')

# apply the function to data frame: 
dataJail[titlesNew[1:]]=dataJail[titlesNew[1:]].apply(toNum)

In [ ]:
# this is better:
dataJail.dtypes

Now, I want to know the population per county. These data do not have that information. I need to get information from another file. You can find it in this [website](http://www.ofm.wa.gov/pop/april1/) (the file is "April 1, 2016 population of cities, towns, and counties used for the allocation of selected state revenues"). This file is in Excel:

In [ ]:
dataLink='http://www.ofm.wa.gov/pop/april1/ofm_april1_population_final.xlsx'
dataPOP=pd.read_excel(dataLink,0)
dataPOP.head(20)

Let's reopen again, skipping the first four rows, and keeping 2, 3, 4, and 11th column:

In [ ]:
dataPOP=pd.read_excel(dataLink,0,skiprows=4)
dataPOP=dataPOP.iloc[:,[1,2,3,10]]

In [ ]:
dataPOP.head(20)

These data have a friendly column named _Filter_, which helps knowing the highest administrative area population. As we want the population per county, we would just select the rows with value _1_ in the filter:

In [ ]:
# this is the strategy
dataPOP[dataPOP.Filter==1]

Does this selection give 39 counties?

In [ ]:
len(dataPOP[dataPOP.Filter==1])

Then, let's subset the original data frame:

In [ ]:
dataPOP=dataPOP[dataPOP.Filter==1]

In [ ]:
# reset the positions:
dataPOP.reset_index(drop=True, inplace=True)
dataPOP.head(10)

What are the data types?

In [ ]:
dataPOP.dtypes


The column with the population estimate is  a string. We need to coerce it to numeric, and rename it with a shorter text:

In [ ]:
dataPOP.rename(columns={"2016 Population Estimate ": "pop2016"},inplace=True)

In [ ]:
dataPOP.iloc[:,3]=pd.to_numeric(dataPOP.iloc[:,3])

In [ ]:
dataPOP.dtypes

Let's show how to integrate these data sets. This will require the use of the function _merge()_.
Merging can integrate two data frames at a time. The main requirement is that both data frames have a column with the same unique values, this is the **key** column. 

Let me subset the _jail_ data just keeping the _county_ and the _admissions_ for 2015:

In [ ]:
jail2015=dataJail.iloc[:,[0,19]]
# you have 
jail2015.head()


Now, let me see dataPOP without the _Filter_ and _Jurisdiction_ column:

In [ ]:
dataPOP=dataPOP.loc[:,['County','pop2016']]
# you get
dataPOP.head()

The candidates for keys are the columns _county_ and _County_, respectively. These are unique and common values in both data frames. However, thet have different capitalization. In this situation, it is always difficult to know if everything should be upper case ot title case, as either may be required later when we keep integrating. In fact, it would be better to have some universal code instead of text as keys. So let me create another column in dataPOP with the uppercase version:

In [ ]:
dataPOP['county']=dataPOP.County.str.upper()

In [ ]:
# merge the data frames, use 'county' for dataJail and 'county' for dataPOP as the key
pd.merge(jail2015,dataPOP, on='county')

Great. As you see, I have information repeated, but in different format. Let me save the result in jail2015:

In [ ]:
jail2015=pd.merge(jail2015,dataPOP, on='county')

We can create a new column, to see what _rate per thousands_ of the current estimated population was recently admitted in jail:

In [ ]:
jail2015['rateJailPop']=jail2015.Admission2015/(jail2015.pop2016/1000)

In [ ]:
jail2015.head()

I could calculate higher order rates, but the county with the minimum population (Garfield) does not go above 10,000 residents, so only thousands are meaningful.

The next important step, specially when you have data that can be represented geographically, is to merge the data with a map. We need to install **geopandas**, please erase momentarily the second '#' below (rewrite it later) and install the latest version.

In [2]:
# You can use pip to install geopandas, but just do this once!!!!!!!

!pip install git+git://github.com/geopandas/geopandas.git

  Cloning git://github.com/geopandas/geopandas.git to /private/var/folders/2n/bkfhfqq16r78g3hf7pdj56y40000gn/T/pip-8_tnzbie-build
    100% |████████████████████████████████| 18.8MB 46kB/s eta 0:00:011
  Using cached cligj-0.4.0-py3-none-any.whl
    100% |████████████████████████████████| 71kB 2.5MB/s ta 0:00:01
  Running setup.py bdist_wheel for munch ... done
  Stored in directory: /Users/JoseManuel/Library/Caches/pip/wheels/6b/a7/c4/fee97ae4038d2e41e1c862f5940237293b613d2dadd078c0b4
Successfully built munch
  Running setup.py install for geopandas ... done


In [3]:
import geopandas as gpd

In [12]:
# getting the map from ALL US counties
# it is a shapefile
import os
folder="data/cb_2015_us_county_20m/"
fileName="cb_2015_us_county_20m.shp"

fileSHP=os.path.join(folder,fileName) 
usaCounties = gpd.read_file(fileSHP)

hr90 = np.array(ps.open(fileSHP.replace('.shp', '.dbf')).by_col('AWATER'))

In [16]:
import pysal as ps
import numpy as np
from pysal.contrib.viz import mapping as maps

maps.plot_choropleth(usaCounties, hr90, type='quantiles', cmap='Greens', figsize=(14, 6))

TypeError: expected str, bytes or os.PathLike object, not GeoDataFrame

This map includes more counties than needed:

In [9]:
usaCounties.dtypes

STATEFP     object
COUNTYFP    object
COUNTYNS    object
AFFGEOID    object
GEOID       object
NAME        object
LSAD        object
ALAND        int64
AWATER       int64
geometry    object
dtype: object

Let's keep the counties from WA

In [ ]:
#%% Subsetting 
waCounties=usaCounties[usaCounties['STATEFP']=='53'] 
waCounties.head()

The column with the county names in title case is the one needed here. I will change that column name

In [ ]:
jail2015 = jail2015.rename(columns={'County': 'NAME'})

Changing the column name will simplify my merge, as the name is the same as the column in the MAP. Let's merge:

In [ ]:
waCountiesMerge =waCounties.merge(jail2015, on='NAME')

In [ ]:
# the result should be fine:
waCountiesMerge.head()

The number or rows is the right one:

In [ ]:
len(waCountiesMerge)

And there is no problem with data types:

In [ ]:
waCountiesMerge.dtypes

We are ready for plotting the map. Jupyter need this command:

In [ ]:
%matplotlib inline

In [ ]:
# input:
varToPlot='rateJailPop'
colorMap='Reds'
numberOfClasses=3
title='Inmates per 1000 habitants'

# organizing input:
args={'column':varToPlot,'cmap':colorMap,'scheme':'Quantiles',"k":numberOfClasses,'figsize':(20,13),'legend':True}

In [ ]:
import matplotlib.pyplot as plt

waCountiesMerge.plot(**args)
plt.title(title,{'fontsize': 20})

Since you be publishin online, you may like to explore **folium**.

Folium requires that we transform our current geodataframe into geojson, let's so that:

In [ ]:
# to_crs(epsg='4326') is needed for the map projection:
waGeoJSON = waCountiesMerge.to_crs(epsg='4326').to_json()

Before we used 3 breaks using a quantile fashion; we need to give the values of the intervals to folium:

In [ ]:
# this is what you need:
pd.qcut(waCountiesMerge['rateJailPop'],3).cat.categories

To facilitate getting those values, we add the argument **netbins** and unpack:

In [ ]:
categories, edges = pd.qcut(waCountiesMerge['rateJailPop'],3, retbins=True)

In [ ]:
# this is what we need!
edges

We also need the center of the map, we get it like this:

In [ ]:
from statistics import mean

Longs=[e.x for e in waCountiesMerge['geometry'].centroid]
Lats=[e.y for e in waCountiesMerge['geometry'].centroid]
mapCenter=[mean(Lats),mean(Longs)]

In [ ]:
import folium
map = folium.Map(mapCenter,zoom_start=7)
map.choropleth(geo_str=waGeoJSON, 
               data=waCountiesMerge,columns=['NAME','rateJailPop'],key_on='feature.properties.NAME',
               threshold_scale=list(edges),
               fill_color='PuRd', fill_opacity=0.7, line_opacity=0.5,
               legend_name='Density', reset=True)
map

### Homework:

Integrate your data sets, and include a map in the merging process.